In [2]:
!pip install  -q kaggle

In [16]:
from google.colab import files
import os

# Upload the file
uploaded = files.upload()

# Rename the uploaded file
for filename in uploaded.keys():
    os.rename(filename, 'kaggle.json')  # Rename the file to 'kaggle'


Saving kaggle (3).json to kaggle (3) (1).json


In [18]:
! mkdir ~/.kaggle

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [19]:
! cp kaggle.json ~/.kaggle

In [20]:
! chmod 600 ~/.kaggle/kaggle.json

In [21]:
!kaggle datasets download -d jaimebadiola/177-million-bitcoin-tweets

 99% 1.58G/1.59G [00:18<00:00, 111MB/s]
100% 1.59G/1.59G [00:18<00:00, 92.4MB/s]


In [26]:
!unzip  177-million-bitcoin-tweets.zip

Archive:  177-million-bitcoin-tweets.zip
  inflating: Raw-20201031T194820Z-001/Raw/Abril 2018/2018-04-07.csv  
  inflating: Raw-20201031T194820Z-001/Raw/Abril 2018/2018-04-11.csv  
  inflating: Raw-20201031T194820Z-001/Raw/Abril 2018/2018-04-13.csv  
  inflating: Raw-20201031T194820Z-001/Raw/Abril 2018/2018-04-14.csv  
  inflating: Raw-20201031T194820Z-001/Raw/Abril 2018/2018-04-15.csv  
  inflating: Raw-20201031T194820Z-001/Raw/Abril 2018/2018-04-17.csv  
  inflating: Raw-20201031T194820Z-001/Raw/Abril 2018/2018-04-18.csv  
  inflating: Raw-20201031T194820Z-001/Raw/Abril 2018/2018-04-19.csv  
  inflating: Raw-20201031T194820Z-001/Raw/Abril 2018/2018-04-20.csv  
  inflating: Raw-20201031T194820Z-001/Raw/Abril 2018/2018-04-21.csv  
  inflating: Raw-20201031T194820Z-001/Raw/Abril 2018/2018-04-22.csv  
  inflating: Raw-20201031T194820Z-001/Raw/Abril 2018/2018-04-23.csv  
  inflating: Raw-20201031T194820Z-001/Raw/Abril 2018/2018-04-24.csv  
  inflating: Raw-20201031T194820Z-001/Raw/Abril 2

In [23]:
!pip install pyspark

In [27]:
from pyspark.sql.functions import input_file_name
from pyspark.sql import SparkSession
from pyspark.sql.types import *

# Create SparkSession
spark = SparkSession.builder \
    .appName("Read CSV Files") \
    .getOrCreate()


In [28]:
# Define custom schema
customSchema = StructType([
    StructField("userName", StringType(), True),
    StructField("date", DateType(), True),
    StructField("#retweets", IntegerType(), True),
    StructField("#favorites", StringType(), True),
    StructField("text", StringType(), True),
    StructField("geo", StringType(), True),
    StructField("#mentions", StringType(), True),
    StructField("#hashtags", StringType(), True),
    StructField("id", LongType(), True),
    StructField("permalink", StringType(), True)
])

# Function to read CSV files and return a DataFrame
def read_csv_files(path):
    return spark.read.format("csv") \
        .option("header", "true") \
        .option("sep", ";") \
        .schema(customSchema) \
        .load(path) \
        .withColumn("filename", input_file_name())

# Paths to the directories containing CSV files
paths = [
    'Raw-20201031T194820Z-001/*/*/*.csv',
     'Raw-20201031T194820Z-002/*/*/*.csv',
    'Raw-20201031T194820Z-003/*/*/*.csv',
    # Add paths to other directories here
]

# List to hold DataFrames
dfs = []

# Read CSV files and append DataFrames
for path in paths:
    df = read_csv_files(path)
    dfs.append(df)
from functools import reduce
# Combine all DataFrames into one
combined_df = reduce(lambda df1, df2: df1.union(df2), dfs)

# Show the DataFrame schema and first few rows
combined_df.printSchema()
combined_df.show()

root
 |-- userName: string (nullable = true)
 |-- date: date (nullable = true)
 |-- #retweets: integer (nullable = true)
 |-- #favorites: string (nullable = true)
 |-- text: string (nullable = true)
 |-- geo: string (nullable = true)
 |-- #mentions: string (nullable = true)
 |-- #hashtags: string (nullable = true)
 |-- id: long (nullable = true)
 |-- permalink: string (nullable = true)
 |-- filename: string (nullable = false)

+---------------+----------+---------+----------+--------------------+----+--------------------+--------------------+------------------+--------------------+--------------------+
|       userName|      date|#retweets|#favorites|                text| geo|           #mentions|           #hashtags|                id|           permalink|            filename|
+---------------+----------+---------+----------+--------------------+----+--------------------+--------------------+------------------+--------------------+--------------------+
|   stacyherbert|2017-12-07|    

In [29]:
combined_df.count()

17792925

In [35]:
combined_df.select("text").show()

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/usr/local/lib/python3.10/dist-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/usr/lib/python3.10/socket.py", line 705, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 